In [1]:
import pandas as pd
import json
from itertools import chain

files = [
    "js_2023_02_06/warm/query_responses.json",
    "js_2023_02_06/cold/query_responses.json",
    # "irmin+1669935078/merlin_responses.json",
]


def frame_from_file(file):
    data = pd.read_json("js_2023_02_06/warm/query_responses.json", lines = True)

    resp = list(chain.from_iterable(data['responses']))

    resp_with_id = []

    for _, row in data.iterrows():
        sample_id = row['sample_id']
        for resp in row['responses']:
            resp_copy = resp.copy()
            resp_copy['sample_id'] = sample_id
            resp_with_id.append(resp_copy)

    return pd.json_normalize(resp_with_id, sep='.')


In [2]:
def print_percentiles(frame, file):
    percentiles = [0.25, 0.5, 0.75, 0.99]


    desc_clock = frame['timing.clock'].describe(percentiles=percentiles)
    desc_cpu = frame['timing.cpu'].describe(percentiles=percentiles)
    desc_reader = frame['timing.reader'].describe(percentiles=percentiles)
    desc_ppx = frame['timing.ppx'].describe(percentiles=percentiles)
    desc_typer = frame['timing.typer'].describe(percentiles=percentiles)
    desc_query = frame['timing.query'].describe(percentiles=percentiles)


    df_clock  = pd.DataFrame(desc_clock).T
    df_cpu    = pd.DataFrame(desc_cpu).T
    df_reader = pd.DataFrame(desc_reader).T
    df_ppx    = pd.DataFrame(desc_ppx).T
    df_typer  = pd.DataFrame(desc_typer).T
    df_query  = pd.DataFrame(desc_query).T


    df_clock.index  = ["timing.clock"]
    df_cpu.index    = ["timing.cpu"]
    df_reader.index = ["timing.reader"]
    df_ppx.index    = ["timing.ppx"]
    df_typer.index  = ["timing.typer"]
    df_query.index  = ["timing.query"]


    print(f"Data from {file}:")
    display(pd.concat([df_clock, df_cpu, df_reader, df_ppx, df_typer, df_query]))
    print("\n\n")

for f in files:
    frame = frame_from_file(f)
    print_percentiles(frame, f)

Data from js_2023_02_06/warm/query_responses.json:


,count,mean,std,min,25%,50%,75%,99%,max
timing.clock,2400.0,88.760833,128.174538,44.0,60.75,80.0,104.00,149.0,3665.0
timing.cpu,2400.0,84.007917,124.314026,43.0,60.00,79.0,94.00,135.0,3650.0
timing.reader,2400.0,13.533750,67.565000,2.0,4.00,10.0,17.00,39.0,3289.0
timing.ppx,2400.0,49.509167,13.388164,32.0,37.00,55.0,63.00,79.0,100.0
timing.typer,2400.0,6.710417,67.783056,2.0,3.00,4.0,9.00,13.0,3249.0
timing.query,2400.0,14.047083,76.726486,0.0,1.00,4.0,16.25,63.0,3563.0





Data from js_2023_02_06/cold/query_responses.json:


,count,mean,std,min,25%,50%,75%,99%,max
timing.clock,2400.0,88.760833,128.174538,44.0,60.75,80.0,104.00,149.0,3665.0
timing.cpu,2400.0,84.007917,124.314026,43.0,60.00,79.0,94.00,135.0,3650.0
timing.reader,2400.0,13.533750,67.565000,2.0,4.00,10.0,17.00,39.0,3289.0
timing.ppx,2400.0,49.509167,13.388164,32.0,37.00,55.0,63.00,79.0,100.0
timing.typer,2400.0,6.710417,67.783056,2.0,3.00,4.0,9.00,13.0,3249.0
timing.query,2400.0,14.047083,76.726486,0.0,1.00,4.0,16.25,63.0,3563.0


In [3]:
sorted = comp_times.sort_values(ascending=False)

sorted.head(20)

NameError: name 'comp_times' is not defined